In [ ]:
try:
    import google.colab  # noqa: F401
except ImportError:
    import ufl  # noqa: F401
    import dolfinx  # noqa: F401
else:
    try:
        import ufl
        import dolfinx
    except ImportError:
        !wget "https://fem-on-colab.github.io/releases/fenicsx-install.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
        import ufl  # noqa: F401
        import dolfinx  # noqa: F401

In [ ]:
try:
    import multiphenicsx
except ImportError:
    !pip3 install git+https://github.com/multiphenics/multiphenicsx.git
    import multiphenicsx  # noqa: F401

In [ ]:
# Download data files
!mkdir -p data
![ -f data/circle.h5 ] || wget https://github.com/multiphenics/multiphenicsx/raw/main/tutorials/03_lagrange_multipliers/data/circle.h5 -O data/circle.h5
![ -f data/circle.xdmf ] || wget https://github.com/multiphenics/multiphenicsx/raw/main/tutorials/03_lagrange_multipliers/data/circle.xdmf -O data/circle.xdmf

In [ ]:
import numpy as np
from mpi4py import MPI
from petsc4py import PETSc
from ufl import grad, inner, Measure, TestFunction, TrialFunction
from dolfinx.fem import DirichletBC, Function, FunctionSpace, LinearProblem, locate_dofs_topological
from dolfinx.mesh import GhostMode
from dolfinx.io import XDMFFile
from multiphenicsx.fem import (assemble_matrix_block, assemble_scalar, assemble_vector_block,
                               BlockVecSubVectorWrapper, create_vector_block, DofMapRestriction)

In [ ]:
with XDMFFile(MPI.COMM_WORLD, "data/circle.xdmf", "r") as infile:
    mesh = infile.read_mesh(GhostMode.none)
    subdomains = infile.read_meshtags(mesh, name="subdomains")
    mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)
    boundaries = infile.read_meshtags(mesh, name="boundaries")
cells_Omega1 = subdomains.indices[subdomains.values == 1]
cells_Omega2 = subdomains.indices[subdomains.values == 2]
facets_partial_Omega = boundaries.indices[boundaries.values == 1]
facets_Gamma = boundaries.indices[boundaries.values == 2]

In [ ]:
# Define associated measures
dx = Measure("dx")(subdomain_data=subdomains)
ds = Measure("ds")(subdomain_data=boundaries)
dS = Measure("dS")(subdomain_data=boundaries)
dS = dS(2)  # restrict to the interface, which has facet ID equal to 2

In [ ]:
# Define function spaces
V = FunctionSpace(mesh, ("Lagrange", 2))
V1 = V.clone()
V2 = V.clone()
M = V.clone()

In [ ]:
# Define restrictions
dofs_V1_Omega1 = locate_dofs_topological(V1, subdomains.dim, cells_Omega1)
dofs_V2_Omega2 = locate_dofs_topological(V2, subdomains.dim, cells_Omega2)
dofs_M_Gamma = locate_dofs_topological(M, boundaries.dim, facets_Gamma)
restriction_V1_Omega1 = DofMapRestriction(V1.dofmap, dofs_V1_Omega1)
restriction_V2_Omega2 = DofMapRestriction(V2.dofmap, dofs_V2_Omega2)
restriction_M_Gamma = DofMapRestriction(M.dofmap, dofs_M_Gamma)
restriction = [restriction_V1_Omega1, restriction_V2_Omega2, restriction_M_Gamma]

In [ ]:
# Define trial and test functions
(u1, u2, l) = (TrialFunction(V1), TrialFunction(V2), TrialFunction(M))
(v1, v2, m) = (TestFunction(V1), TestFunction(V2), TestFunction(M))

In [ ]:
# Define problem block forms
zero = Function(V)
a = [[inner(grad(u1), grad(v1)) * dx(1), None, l("-") * v1("-") * dS],
     [None, inner(grad(u2), grad(v2)) * dx(2), - l("+") * v2("+") * dS],
     [m("-") * u1("-") * dS, - m("+") * u2("+") * dS, None]]
f = [v1 * dx(1), v2 * dx(2), zero * m("-") * dS]

In [ ]:
# Define boundary conditions
dofs_V1_partial_Omega = locate_dofs_topological((V1, V), boundaries.dim, facets_partial_Omega)
dofs_V2_partial_Omega = locate_dofs_topological((V2, V), boundaries.dim, facets_partial_Omega)
bc1 = DirichletBC(zero, dofs_V1_partial_Omega, V1)
bc2 = DirichletBC(zero, dofs_V2_partial_Omega, V2)
bcs = [bc1, bc2]

In [ ]:
# Assemble the block linear system
A = assemble_matrix_block(a, bcs=bcs, restriction=(restriction, restriction))
A.assemble()
F = assemble_vector_block(f, a, bcs=bcs, restriction=restriction)

In [ ]:
# Solve
u1u2l = create_vector_block(f, restriction=restriction)
ksp = PETSc.KSP()
ksp.create(mesh.comm)
ksp.setOperators(A)
ksp.setType("preonly")
ksp.getPC().setType("lu")
ksp.getPC().setFactorSolverType("mumps")
ksp.setFromOptions()
ksp.solve(F, u1u2l)
u1u2l.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

In [ ]:
# Split the block solution in components
(u1, u2, l) = (Function(V1), Function(V2), Function(M))
with BlockVecSubVectorWrapper(u1u2l, [V1.dofmap, V2.dofmap, M.dofmap], restriction) as u1u2l_wrapper:
    for u1u2l_wrapper_local, component in zip(u1u2l_wrapper, (u1, u2, l)):
        with component.vector.localForm() as component_local:
            component_local[:] = u1u2l_wrapper_local

In [ ]:
# Define trial and test functions
u = TrialFunction(V)
v = TestFunction(V)

In [ ]:
# Define problem forms
a_ex = inner(grad(u), grad(v)) * dx
f_ex = v * dx

In [ ]:
# Define Dirichlet BC object on Gamma
dofs_V_partial_Omega = locate_dofs_topological(V, boundaries.dim, facets_partial_Omega)
bc_ex = DirichletBC(zero, dofs_V_partial_Omega)

In [ ]:
# Solve
u_ex = Function(V)
problem_ex = LinearProblem(
    a_ex, f_ex, bcs=[bc_ex], u=u_ex,
    petsc_options={"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"})
problem_ex.solve()
u_ex.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

In [ ]:
u_ex1_norm = np.sqrt(mesh.comm.allreduce(assemble_scalar(inner(u_ex, u_ex) * dx(1)), op=MPI.SUM))
u_ex2_norm = np.sqrt(mesh.comm.allreduce(assemble_scalar(inner(u_ex, u_ex) * dx(2)), op=MPI.SUM))
err1_norm = np.sqrt(mesh.comm.allreduce(assemble_scalar(inner(u_ex - u1, u_ex - u1) * dx(1)), op=MPI.SUM))
err2_norm = np.sqrt(mesh.comm.allreduce(assemble_scalar(inner(u_ex - u2, u_ex - u2) * dx(2)), op=MPI.SUM))
print("Relative error on subdomain 1", err1_norm / u_ex1_norm)
print("Relative error on subdomain 2", err2_norm / u_ex2_norm)
assert np.isclose(err1_norm / u_ex1_norm, 0., atol=1.e-10)
assert np.isclose(err2_norm / u_ex2_norm, 0., atol=1.e-10)

In [ ]:
%%bash

export LD_PRELOAD=""
ERROR_LIBRARIES=($(find /root/.cache/fenics -name '*\.so' -exec \
    bash -c 'ldd $0 | grep libstdc++.so.6 1>/dev/null 2>/dev/null && echo $0' {} \;))
if [ ${#ERROR_LIBRARIES[@]} -eq 0 ]; then
    echo "No reference to libstdc++.so was found"
else
    for ERROR_LIBRARY in "${ERROR_LIBRARIES[@]}"; do
        echo "Error: library $ERROR_LIBRARY depends on libstdc++.so"
        ldd -v $ERROR_LIBRARY
    done
    false
fi